# Word Embeddings
Image and audio processing systems work with rich, high-dimensional datasets encoded as vectors of numbers. However, natural language processing systems traditionally treat words a discrete atomic symbols, and therefore 'cat' may be represented as Id537 and 'dog' as Id143. These encodings are very sparse and provide no useful information regarding the relationships that may exist between the individual symbols. 

Vector space models represent words in a continuous vector space where semantically similar words are mapped to nearby points (are embedded nearby each other). In this series of notebook, we look at few word embedding techniques and compare them:

* Skip-gram with [Negative Sampling](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
* Skip-gram with [Noise Contrastive Estimation](http://papers.nips.cc/paper/5165-learning-word-embeddings-efficiently-with-noise-contrastive-estimation.pdf)
* Glove: [Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf) and more resource from [here](https://nlp.stanford.edu/projects/glove/)

# Skip-gram 
Skip-gram model is to find word representations that are useful for predicting the surrounding words in a sentence or a document. More formally, given a sequence of traning words $w_1,w_2,\ldots,w_T$, the objective of the Skip-gram model is to maximize the average log probability
$$
\frac{1}{T} \sum_{t=1}^T\sum_{-c\leq j \leq c, j\neq 0}\log p(w_{t+j}|w_t)
$$
The Skip-gram defines $p(w_{t+j}|w_t)$ using the softmax function
$$
p(w_{o}|w_{i}) = \frac{\exp\left(u_{w_o}^Tv_{w_{i}}\right)}{\sum_{w=1}^V\exp\left(u_w^Tv_{w_{i}}\right)}
$$
where $V$ is size of vocabulary and
* $w_o$ is output word (outside word or surrounding word)
* $w_i$ is input word (context word or center word)
* $u_w$ is output vector representation
* $v_w$ is input vector representation

This formulation is impractical because the cost of computing the denominator is $O(V)$ where $V$ is often large ($10^5-10^7$).

# Skip-gram with Negative sampling
Mikolov et al. introduce one effecient technique so called Negative sampling (NEG). The NEG re-define the objective as
$$
\log \sigma\left(u_{w_o}^Tv_{w_{i}}\right) + \sum_{i=1}^k \mathbb{E}_{j_i\sim P_n(w)}\log\sigma\left(-u_{j_i}^Tv_{w_{i}}\right)
$$
where
$$
P_n(w) = U(w)^{3/4}/Z
$$
the unigram distribution $U(w)$ raised to the 3/4 power (then normalized by $Z$). The power 3/4 makes less frequent words be sampled more often.

The idea here is to
* maximize the probability that real outside word $w_o$ appears around center word $w_i$
* minimize the probability that random words $j_i$ appears around center word $w_i$

## Implementation planning
Before doing the implementation, we list the required steps
0. Choose dataset: 
    * which corpus to be used for training
    * which test-set to be used for testing
1. Pre-processing raw_tex:
    * extract a set of all words (vocab)
    * map vocab <-> integer id
    * compute words-frequence (we might sub-sampling to remove some frequent words such as 'the,a,an,...e.t.c'), we also need the words-frequence to compute $P_n(w)$
    * convert raw text to list of words-ids
2. Ensemble a graph:
    * Define inputs, targets: must take into account of mini-batches
    * Define trainable variables
    * Define a loss function with randomness $\mathbb{E}_{j_i\sim P_n(w)}\log\sigma\left(-u_{j_i}^Tv_{w_{i}}\right)$
    * Define an optimizer (might need to apply some Gradient-Clipping technique)
3. Training:
    * How to feed inputs/targets data
    * How to measure training performance
    * How to tune hyper-parameters
4. Evaluation:
    * How to measure word2vec quality (hard)
    
## Choose dataset
We use cleaned wiki-dataset from Matt Mahoney's [website](http://mattmahoney.net/dc/textdata.html):
* [text8](http://mattmahoney.net/dc/text8.zip) is small dataset (100Mb) 
* [enwiki9](http://mattmahoney.net/dc/enwik9.zip) is bigger dataset (1Gb)

We use the same script in Matt Mahoney to create text9 data from enwik9.

First we load module for this notebook

In [4]:
import numpy as np
import tensorflow as tf
from time import time

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import sys
if '../common' not in sys.path:
    sys.path.insert(0, '../common')

Note that, the text data is clean text (i.e no punctuation, no new line), let's view first 100 characters of our text-input

In [22]:
text_file = '/home/mvu/workplaces/nlp_datas/text9'
with open(text_file, 'r') as f:
    text = f.read()
    print (text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


## Pre-processing data
We code pre-processing into **Word2VecInput**

In [23]:
from nlp.preprocess_input import Word2VecInput

ts = time()
w2v_input = Word2VecInput(text_file)
print ('Pre-processing took {:.2f} seconds'.format(time() - ts))

Pre-processing took 127.29 seconds


Since pre-processing took quite a long time, we dump pre-processed data into a pickled file which includes vocabs, word2id, id2word, word-frequences and trained_wordids

In [5]:
preprocess_file = '/home/mvu/workplaces/nlp_datas/text9.pkl'

In [27]:
# dump pre-processing data to file
w2v_input.dump(preprocess_file)

## Ensemble a graph
We need to define inputs and targets, re-call that the Skip-gram model is to predict surrounding words given a center word so input will be center word and targets will be surrounding-words. Let's look at an example

In [3]:
from IPython.display import IFrame
IFrame('./skipgram-demo/index.html', width=500, height=750)

We ensemble the graph into **Word2vecNeg** object

In [8]:
import pickle
vocabs, word2id, id2word, freqs, train_wordids = pickle.load(open(preprocess_file, 'rb'))
from nlp.word2vec import Word2vecNeg
w2v_neg = Word2vecNeg(vocabs, word2id, id2word, freqs, train_wordids)

In [11]:
w2v_neg.build_graph()

In [ ]:
w2v_neg.train()

Epoch 1/5 Iteration: 100 Avg. Training loss: 6.4922 0.5315 sec/batch
Epoch 1/5 Iteration: 200 Avg. Training loss: 6.4259 0.5293 sec/batch
Epoch 1/5 Iteration: 300 Avg. Training loss: 6.4940 0.5295 sec/batch
Epoch 1/5 Iteration: 400 Avg. Training loss: 6.4754 0.5477 sec/batch
Epoch 1/5 Iteration: 500 Avg. Training loss: 6.5971 0.5626 sec/batch
Epoch 1/5 Iteration: 600 Avg. Training loss: 6.7659 0.5691 sec/batch
Epoch 1/5 Iteration: 700 Avg. Training loss: 6.6976 0.5680 sec/batch
